In [1]:
%load_ext autoreload
import deepdish as dd
import numpy as np
import pandas as pd
import json
from skimage import io
from notebook_utilities.display import stack_browser, display_array
from matplotlib import pyplot as plt
from fimpy.core.split_dataset import H5SplitDataset

from pathlib import Path

In [21]:
% autoreload
from fimpy.registration.plane import align_single_planes_sobel, displacement_between_planes, shift_planes
from fimpy.registration.volume import sobel_stack
from fimpy.registration.reg_from_skimage import register_translation

In [3]:
path = Path(r"J:\_Shared\exp22_2p\EC\imported\180526 f1\src")
ds = H5SplitDataset(path)
ref = np.mean(ds[:10,:,:,:], 0)
prefilter_sigma = 3.3
upsampling=10

In [4]:
ds.shape

(1226, 80, 200, 410)

In [8]:
whole_stack.shape

(24, 80, 200, 410)

In [11]:
whole_stack = np.zeros((ds.shape[0]//50+1,) + ds.shape[1:])

In [12]:
%%time
whole_stack[:,:,:,:] = ds[:,:,:,:][::50, :,:,:]

Wall time: 1min 29s


In [13]:
ref = whole_stack[:2,:,:,:].mean(0)
sob_ref = sobel_stack(ref, prefilter_sigma)

Find between-planes shifts

In [14]:
shifts_planes = np.zeros((ref.shape[0], 2))
num_planes = ref.shape[0]
centre_plane = int(num_planes // 2)

for i in range(centre_plane, ref.shape[0]-1):
    s, error, diffphase = register_translation(ref[i,:,:], ref[i+1,:,:], 10)
    shifts_planes[i+1,:] = shifts_planes[i,:] + s
for i in range(centre_plane, 0, -1):
    s, error, diffphase = register_translation(ref[i,:,:], ref[i-1,:,:], 10)
    shifts_planes[i-1,:] = shifts_planes[i,:] + s

In [ ]:
display_array(shift_planes(ref[np.newaxis, :,:,:], dict(shifts=shifts_planes)))

In [27]:
shifts_planes[30,:]

array([22.3, -6.9])

In [22]:
i = 30
shifted, shifts = align_single_planes_sobel(whole_stack[:,i:i+1,:,:], np.fft.fftn(sob_ref[i:i+1, :, :]), 
                                   prefilter_sigma=prefilter_sigma, upsample_factor=10, maxshift=15)

In [ ]:
display_array(whole_stack[0,i:i+1,:,:] - whole_stack[-1,i:i+1,:,:])

In [ ]:
display_array(shifted[0,:,:,:] - shifted[-1,:,:,:])